##Project 2: Capstone - Britney Hoang 


####For my final project, I used the Sakila SQL database. The simple buisness process that I will again be inquiring about is the rental sales of each genre at the DVD rental place.

###Import Required Libraries  

In [0]:
import os
import json
import pymongo
import pyspark.pandas as pd  # This uses Koalas that is included in PySpark version 3.2 or newer.
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, BinaryType
from pyspark.sql.types import ByteType, ShortType, IntegerType, LongType, FloatType, DecimalType

### Instantiate Global Variables

In [0]:
# Azure MySQL Server Connection Information ###################
jdbc_hostname = "rcq5jz-mysql.mysql.database.azure.com"
jdbc_port = 3306
src_database = "sakila_dw"

#mongoDB
connection_properties = {
  "user" : "britneyhoang0",
  "password" : "Shawnmendes1",
  "driver" : "org.mariadb.jdbc.Driver"
}

# MongoDB Atlas Connection Information ########################
atlas_cluster_name = "period.vb5iusq"
atlas_database_name = "sakila_dw2"
atlas_user_name = "britneyhoang0"
atlas_password = "Shawnmendes1"

# Data Files (JSON) Information ###############################
dst_database = "sakila_dlh" #new sakila lakehouse
base_dir = "dbfs:/FileStore/ds2002final"
database_dir = f"{base_dir}/{dst_database}"

data_dir = f"{base_dir}/source_data"
batch_dir = f"{data_dir}/batch"
stream_dir = f"{data_dir}/stream"

genres_output_bronze = f"{database_dir}/fact_genres/bronze"
genres_output_silver = f"{database_dir}/fact_genres/silver"
genres_output_gold   = f"{database_dir}/fact_genres/gold"

# Delete the Streaming Files ################################## 
dbutils.fs.rm(f"{database_dir}/fact_genres", True) 

# Delete the Database Files ###################################
dbutils.fs.rm(database_dir, True)


Out[2]: True

### Global functions

In [0]:
# ######################################################################################################################
# Use this Function to Fetch a DataFrame from the MongoDB Atlas database server Using PyMongo.
# ######################################################################################################################
def get_mongo_dataframe(user_id, pwd, cluster_name, db_name, collection, conditions, projection, sort):
    '''Create a client connection to MongoDB'''
    mongo_uri = f"mongodb+srv://{user_id}:{pwd}@{cluster_name}.mongodb.net/{db_name}"
    
    client = pymongo.MongoClient(mongo_uri)

    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = client[db_name]
    if conditions and projection and sort:
        dframe = pd.DataFrame(list(db[collection].find(conditions, projection).sort(sort)))
    elif conditions and projection and not sort:
        dframe = pd.DataFrame(list(db[collection].find(conditions, projection)))
    else:
        dframe = pd.DataFrame(list(db[collection].find()))

    client.close()
    
    return dframe

# ######################################################################################################################
# Use this Function to Create New Collections by Uploading JSON file(s) to the MongoDB Atlas server.
# ######################################################################################################################
def set_mongo_collection(user_id, pwd, cluster_name, db_name, src_file_path, json_files):
    '''Create a client connection to MongoDB'''
    mongo_uri = f"mongodb+srv://{user_id}:{pwd}@{cluster_name}.mongodb.net/{db_name}"
    client = pymongo.MongoClient(mongo_uri)
    db = client[db_name]
    
    '''Read in a JSON file, and Use It to Create a New Collection'''
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(src_file_path, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)

    client.close()
    
    return result

In [0]:
%sql
DROP DATABASE IF EXISTS sakila_dlh CASCADE;

###Get Data from Azure SQL Database

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS sakila_dlh
COMMENT "DS-2002 Final Project Database"
LOCATION "dbfs:/FileStore/ds2002final/sakila_dlh"
WITH DBPROPERTIES (contains_pii = true, purpose = "Final Project");

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW view_date
USING org.apache.spark.sql.jdbc
OPTIONS (
  url "jdbc:mysql://rcq5jz-mysql.mysql.database.azure.com:3306/sakila_dw",
  dbtable "dim_date",
  user "brithoang",
  password "Shawnmendes.1"
)

###Create a New Table that Sources Date Dimension Data from a Table in an Azure MySQL database

In [0]:
%sql
USE DATABASE sakila_dlh;

CREATE OR REPLACE TABLE sakila_dlh.dim_date
COMMENT "Date Dimension Table"
LOCATION "dbfs:/FileStore/ds2002final/sakila_dlh/dim_date"
AS SELECT * FROM view_date

num_affected_rows,num_inserted_rows


In [0]:
%sql
DESCRIBE EXTENDED sakila_dlh.dim_date;

col_name,data_type,comment
date_key,int,null
full_date,date,null
date_name,string,null
date_name_us,string,null
date_name_eu,string,null
day_of_week,int,null
day_name_of_week,string,null
day_of_month,int,null
day_of_year,int,null
weekday_weekend,string,null


In [0]:
%sql
SELECT * FROM sakila_dlh.dim_date LIMIT 5

date_key,full_date,date_name,date_name_us,date_name_eu,day_of_week,day_name_of_week,day_of_month,day_of_year,weekday_weekend,week_of_year,month_name,month_of_year,is_last_day_of_month,calendar_quarter,calendar_year,calendar_year_month,calendar_year_qtr,fiscal_month_of_year,fiscal_quarter,fiscal_year,fiscal_year_month,fiscal_year_qtr
20000101,2000-01-01,2000/01/01,01/01/2000,01/01/2000,7,Saturday,1,1,Weekend,52,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
20000102,2000-01-02,2000/01/02,01/02/2000,02/01/2000,1,Sunday,2,2,Weekend,52,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
20000103,2000-01-03,2000/01/03,01/03/2000,03/01/2000,2,Monday,3,3,Weekday,1,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
20000104,2000-01-04,2000/01/04,01/04/2000,04/01/2000,3,Tuesday,4,4,Weekday,1,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
20000105,2000-01-05,2000/01/05,01/05/2000,05/01/2000,4,Wednesday,5,5,Weekday,1,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3


### Exract Film Dimension from Mongo DB

In [0]:
%scala
import com.mongodb.spark._

val df_film = spark.read.format("com.mongodb.spark.sql.DefaultSource")
.option("database", "sakila").option("collection", "film").load()
.select("film_id", "title", "description", "release_year", "language_id", "original_language_id", "rental_duration", "length", "replacement_cost", "rating",
"special_features", "last_update")

import com.mongodb.spark._
df_film: org.apache.spark.sql.DataFrame = [film_id: int, title: string ... 10 more fields]

In [0]:
%scala
val df_test = df_film.limit(10)
display(df_test)

film_id,title,description,release_year,language_id,original_language_id,rental_duration,length,replacement_cost,rating,special_features,last_update
1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in The Canadian Rockies,2006,1,0,6,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrator And a Explorer who must Find a Car in Ancient China,2006,1,0,3,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42
3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a Car who must Sink a Lumberjack in A Baloon Factory,2006,1,0,7,50,18.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 05:03:42
4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumberjack who must Chase a Monkey in A Shark Tank,2006,1,0,5,117,26.99,G,"Commentaries,Behind the Scenes",2006-02-15 05:03:42
5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And a Dentist who must Pursue a Forensic Psychologist in The Gulf of Mexico,2006,1,0,6,130,22.99,G,Deleted Scenes,2006-02-15 05:03:42
6,AGENT TRUMAN,A Intrepid Panorama of a Robot And a Boy who must Escape a Sumo Wrestler in Ancient China,2006,1,0,3,169,17.99,PG,Deleted Scenes,2006-02-15 05:03:42
7,AIRPLANE SIERRA,A Touching Saga of a Hunter And a Butler who must Discover a Butler in A Jet Boat,2006,1,0,6,62,28.99,PG-13,"Trailers,Deleted Scenes",2006-02-15 05:03:42
8,AIRPORT POLLOCK,A Epic Tale of a Moose And a Girl who must Confront a Monkey in Ancient India,2006,1,0,6,54,15.99,R,Trailers,2006-02-15 05:03:42
9,ALABAMA DEVIL,A Thoughtful Panorama of a Database Administrator And a Mad Scientist who must Outgun a Mad Scientist in A Jet Boat,2006,1,0,3,114,21.99,PG-13,"Trailers,Deleted Scenes",2006-02-15 05:03:42
10,ALADDIN CALENDAR,A Action-Packed Tale of a Man And a Lumberjack who must Reach a Feminist in Ancient China,2006,1,0,6,63,24.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 05:03:42


In [0]:
%scala
df_film.printSchema()

root
-- film_id: integer (nullable = true)
-- title: string (nullable = true)
-- description: string (nullable = true)
-- release_year: string (nullable = true)
-- language_id: integer (nullable = true)
-- original_language_id: integer (nullable = true)
-- rental_duration: integer (nullable = true)
-- length: integer (nullable = true)
-- replacement_cost: double (nullable = true)
-- rating: string (nullable = true)
-- special_features: string (nullable = true)
-- last_update: string (nullable = true)

In [0]:
%scala
df_film.write.format("delta").mode("overwrite").saveAsTable("sakila_dlh.dim_film")

In [0]:
%sql
DESCRIBE EXTENDED sakila_dlh.dim_film

col_name,data_type,comment
film_id,int,null
title,string,null
description,string,null
release_year,string,null
language_id,int,null
original_language_id,int,null
rental_duration,int,null
length,int,null
replacement_cost,double,null
rating,string,null


In [0]:
%sql
SELECT * FROM sakila_dlh.dim_film LIMIT 5

film_id,title,description,release_year,language_id,original_language_id,rental_duration,length,replacement_cost,rating,special_features,last_update
1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in The Canadian Rockies,2006,1,0,6,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrator And a Explorer who must Find a Car in Ancient China,2006,1,0,3,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42
3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a Car who must Sink a Lumberjack in A Baloon Factory,2006,1,0,7,50,18.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 05:03:42
4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumberjack who must Chase a Monkey in A Shark Tank,2006,1,0,5,117,26.99,G,"Commentaries,Behind the Scenes",2006-02-15 05:03:42
5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And a Dentist who must Pursue a Forensic Psychologist in The Gulf of Mexico,2006,1,0,6,130,22.99,G,Deleted Scenes,2006-02-15 05:03:42


###Use PySpark to Read From a CSV File

In [0]:
rental_csv = f"{batch_dir}/rental_table.csv"

df_rental = spark.read.format('csv').options(header='true', inferSchema='true').load(rental_csv)

In [0]:
rent_dis= df_rental.head(15)
display(rent_dis)

rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
1,2005-05-24T22:53:30.000+0000,367,130,2005-05-26T22:04:30.000+0000,1,2006-02-15T21:30:53.000+0000
2,2005-05-24T22:54:33.000+0000,1525,459,2005-05-28T19:40:33.000+0000,1,2006-02-15T21:30:53.000+0000
3,2005-05-24T23:03:39.000+0000,1711,408,2005-06-01T22:12:39.000+0000,1,2006-02-15T21:30:53.000+0000
4,2005-05-24T23:04:41.000+0000,2452,333,2005-06-03T01:43:41.000+0000,2,2006-02-15T21:30:53.000+0000
5,2005-05-24T23:05:21.000+0000,2079,222,2005-06-02T04:33:21.000+0000,1,2006-02-15T21:30:53.000+0000
6,2005-05-24T23:08:07.000+0000,2792,549,2005-05-27T01:32:07.000+0000,1,2006-02-15T21:30:53.000+0000
7,2005-05-24T23:11:53.000+0000,3995,269,2005-05-29T20:34:53.000+0000,2,2006-02-15T21:30:53.000+0000
8,2005-05-24T23:31:46.000+0000,2346,239,2005-05-27T23:33:46.000+0000,2,2006-02-15T21:30:53.000+0000
9,2005-05-25T00:00:40.000+0000,2580,126,2005-05-28T00:22:40.000+0000,1,2006-02-15T21:30:53.000+0000
10,2005-05-25T00:02:21.000+0000,1824,399,2005-05-31T22:44:21.000+0000,2,2006-02-15T21:30:53.000+0000


In [0]:
#prints the files it currently sees in the batch directory
display(dbutils.fs.ls(batch_dir))

path,name,size,modificationTime
dbfs:/FileStore/ds2002final/source_data/batch/rental_table.csv,rental_table.csv,80552,1701307307000


In [0]:
df_rental.write.format("delta").mode("overwrite").saveAsTable("sakila_dlh.dim_rental")

In [0]:
%sql
DESCRIBE EXTENDED sakila_dlh.dim_rental;

col_name,data_type,comment
rental_id,int,null
rental_date,timestamp,null
inventory_id,int,null
customer_id,int,null
return_date,timestamp,null
staff_id,int,null
last_update,timestamp,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,


In [0]:
%sql
SELECT * FROM sakila_dlh.dim_rental LIMIT 5;

rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
1,2005-05-24T22:53:30.000+0000,367,130,2005-05-26T22:04:30.000+0000,1,2006-02-15T21:30:53.000+0000
2,2005-05-24T22:54:33.000+0000,1525,459,2005-05-28T19:40:33.000+0000,1,2006-02-15T21:30:53.000+0000
3,2005-05-24T23:03:39.000+0000,1711,408,2005-06-01T22:12:39.000+0000,1,2006-02-15T21:30:53.000+0000
4,2005-05-24T23:04:41.000+0000,2452,333,2005-06-03T01:43:41.000+0000,2,2006-02-15T21:30:53.000+0000
5,2005-05-24T23:05:21.000+0000,2079,222,2005-06-02T04:33:21.000+0000,1,2006-02-15T21:30:53.000+0000


###Create the rest of tables from Azure MySQL database

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW view_inventory
USING org.apache.spark.sql.jdbc
OPTIONS (
  url "jdbc:mysql://rcq5jz-mysql.mysql.database.azure.com:3306/sakila_dw",
  dbtable "dim_inventory",
  user "brithoang",
  password "Shawnmendes.1"
)

In [0]:
%sql
USE DATABASE sakila_dlh;

CREATE OR REPLACE TABLE sakila_dlh.dim_inventory
COMMENT "Inventory Dimension Table"
LOCATION "dbfs:/FileStore/source_data/sakila_dlh/dim_inventory"
AS SELECT * FROM view_inventory

-- Create a new table named "northwind_dlh.dim_product" using data from the view named "view_product"

num_affected_rows,num_inserted_rows


In [0]:
%sql
DESCRIBE EXTENDED sakila_dlh.dim_inventory;

col_name,data_type,comment
inventory_key,bigint,null
film_key,bigint,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,sakila_dlh,
Table,dim_inventory,
Created Time,Tue Dec 05 05:18:04 UTC 2023,
Last Access,UNKNOWN,
Created By,Spark 3.3.0,


In [0]:
%sql
SELECT * FROM sakila_dlh.dim_inventory LIMIT 5

inventory_key,film_key
1,1
2,1
3,1
4,1
5,1


In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW view_film_category
USING org.apache.spark.sql.jdbc
OPTIONS (
  url "jdbc:mysql://rcq5jz-mysql.mysql.database.azure.com:3306/sakila_dw",
  dbtable "dim_film_category",
  user "brithoang",
  password "Shawnmendes.1"
)

In [0]:
%sql
USE DATABASE sakila_dlh;

CREATE OR REPLACE TABLE sakila_dlh.dim_film_category
COMMENT "Film Category Dimension Table"
LOCATION "dbfs:/FileStore/ds2002-final/sakila_dlh/dim_film_category"
AS SELECT * FROM view_film_category

num_affected_rows,num_inserted_rows


In [0]:
%sql
DESCRIBE EXTENDED sakila_dlh.dim_film_category;

col_name,data_type,comment
film_category_key,bigint,null
film_key,bigint,null
category_key,bigint,null
last_update,timestamp,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,sakila_dlh,
Table,dim_film_category,
Created Time,Tue Dec 05 05:18:07 UTC 2023,


In [0]:
%sql
SELECT * FROM sakila_dlh.dim_film_category LIMIT 5

film_category_key,film_key,category_key,last_update
1,1,6,2006-02-15T05:07:09.000+0000
2,2,11,2006-02-15T05:07:09.000+0000
3,3,6,2006-02-15T05:07:09.000+0000
4,4,11,2006-02-15T05:07:09.000+0000
5,5,8,2006-02-15T05:07:09.000+0000


In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW view_category
USING org.apache.spark.sql.jdbc
OPTIONS (
  url "jdbc:mysql://rcq5jz-mysql.mysql.database.azure.com:3306/sakila_dw",
  dbtable "dim_category",
  user "brithoang",
  password "Shawnmendes.1"
)

In [0]:
%sql
USE DATABASE sakila_dlh;

CREATE OR REPLACE TABLE sakila_dlh.dim_category
COMMENT "Category Dimension Table"
LOCATION "dbfs:/FileStore/ds2002-final/sakila_dlh/dim_category"
AS SELECT * FROM view_category

num_affected_rows,num_inserted_rows


In [0]:
%sql
DESCRIBE EXTENDED sakila_dlh.dim_category;

col_name,data_type,comment
category_key,bigint,null
name,string,null
last_update,timestamp,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,sakila_dlh,
Table,dim_category,
Created Time,Tue Dec 05 05:18:09 UTC 2023,
Last Access,UNKNOWN,


In [0]:
%sql
SELECT * FROM sakila_dlh.dim_category LIMIT 5

category_key,name,last_update
1,Action,2006-02-15T04:46:27.000+0000
2,Animation,2006-02-15T04:46:27.000+0000
3,Children,2006-02-15T04:46:27.000+0000
4,Classics,2006-02-15T04:46:27.000+0000
5,Comedy,2006-02-15T04:46:27.000+0000


### Verify Dimension Tables

In [0]:
%sql
USE sakila_dlh;
SHOW TABLES

database,tableName,isTemporary
sakila_dlh,dim_category,false
sakila_dlh,dim_date,false
sakila_dlh,dim_film,false
sakila_dlh,dim_film_category,false
sakila_dlh,dim_inventory,false
sakila_dlh,dim_rental,false
,view_category,true
,view_date,true
,view_film_category,true
,view_inventory,true


### Use AutoLoader to Process Streaming (Hot Path) Orders Fact Data 
###Bronze Table: Process 'Raw' JSON Data

In [0]:
display(dbutils.fs.ls(stream_dir))

path,name,size,modificationTime
dbfs:/FileStore/ds2002final/source_data/stream/fact_gen1.json,fact_gen1.json,87148,1701378212000
dbfs:/FileStore/ds2002final/source_data/stream/fact_gen2.json,fact_gen2.json,87455,1701378212000
dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json,fact_gen3.json,87684,1701378212000


In [0]:
(spark.readStream
 .format("cloudFiles")
 .option("cloudFiles.format", "json")
 #.option("cloudFiles.schemaHints", "date_key BIGINT")
 #.option("cloudFiles.schemaHints", "rental_key BIGINT")
 #.option("cloudFiles.schemaHints", "last_name STRING")
 #.option("cloudFiles.schemaHints", "inventory_key STRING")
 #.option("cloudFiles.schemaHints", "film_category_key STRING")
 #.option("cloudFiles.schemaHints", "category_key STRING")
 #.option("cloudFiles.schemaHints", "release_year BIGINT")
 #.option("cloudFiles.schemaHints", "rental_date BIGINT")
 #.option("cloudFiles.schemaHints", "title STRING")
 #.option("cloudFiles.schemaHints", "rating STRING")
 #.option("cloudFiles.schemaHints", "name STRING")
 #.option("cloudFiles.schemaHints", "rental_key BIGINT")
 .option("cloudFiles.schemaLocation", genres_output_bronze)
 .option("cloudFiles.inferColumnTypes", "true")
 .option("multiLine", "true")
 .load(stream_dir)
 .createOrReplaceTempView("genres_raw_tempview"))

In [0]:
%sql
/* Add Metadata for Traceability */
CREATE OR REPLACE TEMPORARY VIEW genres_bronze_tempview AS (
  SELECT *, current_timestamp() receipt_time, input_file_name() source_file
  FROM genres_raw_tempview
)

In [0]:
%sql
SELECT * FROM genres_bronze_tempview LIMIT 15

category_key,fact_genre_key,film_category_key,film_key,genre,inventory_key,rating,release_year,rental_date,rental_key,title,_rescued_data,receipt_time,source_file
7,601,585,585,Drama,2665,G,2006,2005-05-26 21:22:07,305,MOB DUFFEL,null,2023-12-05T05:18:13.674+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json
1,602,586,586,Action,2671,PG,2006,2005-05-27 19:03:08,448,MOCKINGBIRD HOLLYWOOD,null,2023-12-05T05:18:13.674+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json
3,603,588,588,Children,2679,NC-17,2006,2005-05-28 11:48:55,583,MODEL FISH,null,2023-12-05T05:18:13.674+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json
3,604,588,588,Children,2678,NC-17,2006,2005-05-27 01:18:57,327,MODEL FISH,null,2023-12-05T05:18:13.674+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json
6,605,589,589,Documentary,2685,PG,2006,2005-05-28 18:24:43,638,MODERN DORADO,null,2023-12-05T05:18:13.674+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json
6,606,589,589,Documentary,2684,PG,2006,2005-05-27 10:35:27,387,MODERN DORADO,null,2023-12-05T05:18:13.674+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json
12,607,592,592,Music,2697,PG,2006,2005-05-29 16:59:44,796,MONSTER SPARTACUS,null,2023-12-05T05:18:13.674+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json
11,608,593,593,Horror,2701,G,2006,2005-05-25 00:31:15,14,MONTEREY LABYRINTH,null,2023-12-05T05:18:13.674+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json
9,609,595,595,Foreign,2711,PG,2006,2005-05-29 09:55:43,751,MOON BUNCH,null,2023-12-05T05:18:13.674+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json
9,610,595,595,Foreign,2714,PG,2006,2005-05-26 03:07:43,168,MOON BUNCH,null,2023-12-05T05:18:13.674+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json


In [0]:
(spark.table("genres_bronze_tempview")
      .writeStream
      .format("delta")
      .option("checkpointLocation", f"{genres_output_bronze}/_checkpoint")
      .outputMode("append")
      .table("fact_genres_bronze"))

Out[35]: <pyspark.sql.streaming.query.StreamingQuery at 0x7f3acdc421c0>

###Silver Table: Include Reference Data

In [0]:
(spark.readStream
  .table("fact_genres_bronze")
  .createOrReplaceTempView("genres_silver_tempview"))

In [0]:
%sql
SELECT * FROM genres_silver_tempview LIMIT 15

category_key,fact_genre_key,film_category_key,film_key,genre,inventory_key,rating,release_year,rental_date,rental_key,title,_rescued_data,receipt_time,source_file
7,601,585,585,Drama,2665,G,2006,2005-05-26 21:22:07,305,MOB DUFFEL,null,2023-12-05T05:18:14.633+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json
1,602,586,586,Action,2671,PG,2006,2005-05-27 19:03:08,448,MOCKINGBIRD HOLLYWOOD,null,2023-12-05T05:18:14.633+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json
3,603,588,588,Children,2679,NC-17,2006,2005-05-28 11:48:55,583,MODEL FISH,null,2023-12-05T05:18:14.633+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json
3,604,588,588,Children,2678,NC-17,2006,2005-05-27 01:18:57,327,MODEL FISH,null,2023-12-05T05:18:14.633+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json
6,605,589,589,Documentary,2685,PG,2006,2005-05-28 18:24:43,638,MODERN DORADO,null,2023-12-05T05:18:14.633+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json
6,606,589,589,Documentary,2684,PG,2006,2005-05-27 10:35:27,387,MODERN DORADO,null,2023-12-05T05:18:14.633+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json
12,607,592,592,Music,2697,PG,2006,2005-05-29 16:59:44,796,MONSTER SPARTACUS,null,2023-12-05T05:18:14.633+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json
11,608,593,593,Horror,2701,G,2006,2005-05-25 00:31:15,14,MONTEREY LABYRINTH,null,2023-12-05T05:18:14.633+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json
9,609,595,595,Foreign,2711,PG,2006,2005-05-29 09:55:43,751,MOON BUNCH,null,2023-12-05T05:18:14.633+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json
9,610,595,595,Foreign,2714,PG,2006,2005-05-26 03:07:43,168,MOON BUNCH,null,2023-12-05T05:18:14.633+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json


In [0]:
%sql
DESCRIBE EXTENDED genres_silver_tempview

col_name,data_type,comment
category_key,bigint,null
fact_genre_key,bigint,null
film_category_key,bigint,null
film_key,bigint,null
genre,string,null
inventory_key,bigint,null
rating,string,null
release_year,string,null
rental_date,string,null
rental_key,bigint,null


In [0]:
%sql
SELECT * FROM genres_silver_tempview LIMIT 15

category_key,fact_genre_key,film_category_key,film_key,genre,inventory_key,rating,release_year,rental_date,rental_key,title,_rescued_data,receipt_time,source_file
7,601,585,585,Drama,2665,G,2006,2005-05-26 21:22:07,305,MOB DUFFEL,null,2023-12-05T05:18:14.633+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json
1,602,586,586,Action,2671,PG,2006,2005-05-27 19:03:08,448,MOCKINGBIRD HOLLYWOOD,null,2023-12-05T05:18:14.633+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json
3,603,588,588,Children,2679,NC-17,2006,2005-05-28 11:48:55,583,MODEL FISH,null,2023-12-05T05:18:14.633+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json
3,604,588,588,Children,2678,NC-17,2006,2005-05-27 01:18:57,327,MODEL FISH,null,2023-12-05T05:18:14.633+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json
6,605,589,589,Documentary,2685,PG,2006,2005-05-28 18:24:43,638,MODERN DORADO,null,2023-12-05T05:18:14.633+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json
6,606,589,589,Documentary,2684,PG,2006,2005-05-27 10:35:27,387,MODERN DORADO,null,2023-12-05T05:18:14.633+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json
12,607,592,592,Music,2697,PG,2006,2005-05-29 16:59:44,796,MONSTER SPARTACUS,null,2023-12-05T05:18:14.633+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json
11,608,593,593,Horror,2701,G,2006,2005-05-25 00:31:15,14,MONTEREY LABYRINTH,null,2023-12-05T05:18:14.633+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json
9,609,595,595,Foreign,2711,PG,2006,2005-05-29 09:55:43,751,MOON BUNCH,null,2023-12-05T05:18:14.633+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json
9,610,595,595,Foreign,2714,PG,2006,2005-05-26 03:07:43,168,MOON BUNCH,null,2023-12-05T05:18:14.633+0000,dbfs:/FileStore/ds2002final/source_data/stream/fact_gen3.json


In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW fact_genres_silver_tempview AS (
  SELECT r.rental_id,
      r.rental_date, 
      f.film_id,
      f.title,
      f.rating,
      f.release_year,
      fc.film_category_key,
      c.category_key,
      c.name as genre,
      gs.inventory_key
  FROM genres_silver_tempview as gs
  INNER JOIN sakila_dlh.dim_inventory as i 
  ON gs.inventory_key = i.inventory_key
  INNER JOIN sakila_dlh.dim_rental as r
  ON r.inventory_id = i.inventory_key
  INNER JOIN sakila_dlh.dim_film as f
  ON i.film_key = f.film_id
  INNER JOIN sakila_dlh.dim_film_category as fc
  ON f.film_id = fc.film_key
  INNER JOIN sakila_dlh.dim_category as c
  ON fc.category_key = c.category_key
)

##### Even though I uploaded the dim_date dimension, I did not include it in my silver table because I will be using the rental date in order to calculate my solution. Thus, it is not seem appropriate to add in the dim_date because it doesn't make sense in regards to my project/overall question. 

In [0]:
(spark.table("fact_genres_silver_tempview")
      .writeStream
      .format("delta")
      .option("checkpointLocation", f"{genres_output_silver}_checkpoint")
      .outputMode("append")
      .table("fact_genres_silver"))

Out[57]: <pyspark.sql.streaming.query.StreamingQuery at 0x7f3acdc42df0>

In [0]:
%sql
SELECT * FROM fact_genres_silver LIMIT 15

rental_id,rental_date,film_id,title,rating,release_year,film_category_key,category_key,genre,inventory_key
972,2005-05-30T20:21:07.000+0000,1,ACADEMY DINOSAUR,PG,2006,1,6,Documentary,2
361,2005-05-27T07:03:28.000+0000,1,ACADEMY DINOSAUR,PG,2006,1,6,Documentary,6
995,2005-05-31T00:06:02.000+0000,4,AFFAIR PREJUDICE,G,2006,4,11,Horror,17
465,2005-05-27T20:44:36.000+0000,4,AFFAIR PREJUDICE,G,2006,4,11,Horror,20
552,2005-05-28T07:53:38.000+0000,5,AFRICAN EGG,G,2006,5,8,Family,23
317,2005-05-26T23:23:56.000+0000,6,AGENT TRUMAN,PG,2006,6,9,Foreign,26
259,2005-05-26T15:32:46.000+0000,6,AGENT TRUMAN,PG,2006,6,9,Foreign,30
120,2005-05-25T19:37:47.000+0000,8,AIRPORT POLLOCK,R,2006,8,11,Horror,37
785,2005-05-29T15:08:41.000+0000,8,AIRPORT POLLOCK,R,2006,8,11,Horror,39
563,2005-05-28T09:10:49.000+0000,11,ALAMO VIDEOTAPE,G,2006,11,9,Foreign,53


In [0]:
%sql
DESCRIBE EXTENDED sakila_dlh.fact_genres_silver

col_name,data_type,comment
rental_id,int,null
rental_date,timestamp,null
film_id,int,null
title,string,null
rating,string,null
release_year,string,null
film_category_key,bigint,null
category_key,bigint,null
genre,string,null
inventory_key,bigint,null


### Gold Table: Perform Aggregations
###Create a new Gold table using the CTAS approach. 

In [0]:
(spark.readStream
  .table("fact_genres_silver")
  .createOrReplaceTempView("genres_gold_tempview"))

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW genres_gold_tempview AS (
SELECT title, 
genre as Film_Genre,
category_key,
release_year,
COUNT ('rental_date)') as Rental_Count
FROM sakila_dlh.fact_genres_silver
GROUP BY title, Film_Genre, category_key, release_year
ORDER BY Rental_Count DESC
)

#### I used the rental date in order to calculate how many times each movie was rented in order to look at the genres. With this, people can analyze the most popular movies and genres that are rented out. 

In [0]:
%sql
SELECT * FROM genres_gold_tempview LIMIT 10

title,Film_Genre,category_key,release_year,Rental_Count
ROBBERS JOON,Children,3,2006,5
RIDGEMONT SUBMARINE,New,13,2006,4
NETWORK PEAK,Family,8,2006,4
ROSES TREASURE,Sports,15,2006,4
ROCKETEER MOTHER,Foreign,9,2006,4
GRAFFITI LOVE,Sci-Fi,14,2006,4
PACIFIC AMISTAD,Documentary,6,2006,4
FORWARD TEMPLE,Games,10,2006,4
SEATTLE EXPECATIONS,Drama,7,2006,4
TIMBERLAND SKY,Classics,4,2006,4


In [0]:
(spark.table("genres_gold_tempview")
      .write
      .format("delta")
      .option("path", f"{genres_output_gold}/fact_genres_gold")
      .option("checkpointLocation", f"{genres_output_gold}/_checkpoint")
      .mode("overwrite")
      .saveAsTable("fact_genres_gold"))

In [0]:
%sql
SELECT * FROM fact_genres_gold LIMIT 15

title,Film_Genre,category_key,release_year,Rental_Count
ROBBERS JOON,Children,3,2006,5
BOOGIE AMELIE,Music,12,2006,4
PULP BEVERLY,Horror,11,2006,4
GRAFFITI LOVE,Sci-Fi,14,2006,4
FANTASY TROOPERS,Action,1,2006,4
FORWARD TEMPLE,Games,10,2006,4
HILLS NEIGHBORS,Action,1,2006,4
ROSES TREASURE,Sports,15,2006,4
NETWORK PEAK,Family,8,2006,4
ROCKETEER MOTHER,Foreign,9,2006,4


In [0]:
%sql
DESCRIBE EXTENDED fact_genres_gold

col_name,data_type,comment
title,string,null
Film_Genre,string,null
category_key,bigint,null
release_year,string,null
Rental_Count,bigint,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,sakila_dlh,
Table,fact_genres_gold,


### Query the Results
### Visualization

#### This visualization demonstrates the buinsess value of my solution because owners could look at this data and see which movies tend to do well in the store. Thus, owners can purchase more of that genre (such as Family movies) to increase profit and also save money by not buying less popular genres (such as Horror films).

In [0]:
%sql
SELECT * FROM fact_genres_gold

title,Film_Genre,category_key,release_year,Rental_Count
ROBBERS JOON,Children,3,2006,5
BOOGIE AMELIE,Music,12,2006,4
PULP BEVERLY,Horror,11,2006,4
GRAFFITI LOVE,Sci-Fi,14,2006,4
FANTASY TROOPERS,Action,1,2006,4
FORWARD TEMPLE,Games,10,2006,4
HILLS NEIGHBORS,Action,1,2006,4
ROSES TREASURE,Sports,15,2006,4
NETWORK PEAK,Family,8,2006,4
ROCKETEER MOTHER,Foreign,9,2006,4


Databricks visualization. Run in Databricks to view.

### Clean up the File System 

In [0]:
#%fs rm -r /FileStore/ds2002final/